In [1]:
#notebook for exploring and preprocessing the NYT bestseller list dataset
# obtained from https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers/version/1
# originally downloaded from NYT
# there is a kernel associated with this dataset, I use some methods from it
# the use of Levenshtein distance for checking integrity is my work

import json
import pandas as pd
import numpy as np

import re

from string import ascii_lowercase
from datetime import datetime

import Levenshtein as lev



In [ ]:
#import sys
#!{sys.executable} -m pip install python-Levenshtein

In [2]:
file = 'nyt2.json'
nyt_best = pd.read_json(file, lines=True, orient='columns')
nyt_best.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,{'$oid': '5b4aa4ead3089013507db18b'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",{'$numberInt': '27'},Bantam,ODD HOURS,{'$numberInt': '1'},{'$numberInt': '0'},{'$numberInt': '1'}
1,{'$oid': '5b4aa4ead3089013507db18c'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,{'$numberInt': '2'},{'$numberInt': '1'},{'$numberInt': '3'}
2,{'$oid': '5b4aa4ead3089013507db18d'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin's,LOVE THE ONE YOU'RE WITH,{'$numberInt': '3'},{'$numberInt': '2'},{'$numberInt': '2'}
3,{'$oid': '5b4aa4ead3089013507db18e'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,{'$numberDouble': '22.95'},Putnam,THE FRONT,{'$numberInt': '4'},{'$numberInt': '0'},{'$numberInt': '1'}
4,{'$oid': '5b4aa4ead3089013507db18f'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,{'$numberDouble': '24.95'},Doubleday,SNUFF,{'$numberInt': '5'},{'$numberInt': '0'},{'$numberInt': '1'}


In [3]:
#following is from kaggle kernel
# https://www.kaggle.com/tobaotic/clean-data



#convert string to lowercase and replace all non alphanumeric characters
def get_lc_values(v):
    regex = re.compile('[^a-z0-9]')
    lc_values = []
    for x in v:
        #this strips away all whitespace and makes everything lowercase
        lc_values.append(regex.sub('', x.lower()))

    return lc_values



def flatten_json(y):
    # extract values from json string in dataframe
    
    out = {}
    ret_val = '-'
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        elif type(x) is str:
            out[name[:-1]] = x


    flatten(y)

    if type(out) is dict:
        for v in out:
            ret_val = out[v]
    
    return ret_val


In [4]:
#also from kernel

for i in range(len(nyt_best)):
    nyt_best.loc[i]["_id"] = flatten_json(nyt_best.loc[i]["_id"])
    nyt_best.loc[i]["bestsellers_date"] = datetime.fromtimestamp(int(flatten_json(nyt_best.loc[i]["bestsellers_date"])[:10])).strftime('%Y-%m-%d')
    nyt_best.loc[i]["price"] = flatten_json(nyt_best.loc[i]["price"])
    nyt_best.loc[i]["published_date"] = datetime.fromtimestamp(int(flatten_json(nyt_best.loc[i]["published_date"])[:10])).strftime('%Y-%m-%d')
    nyt_best.loc[i]["rank"] = flatten_json(nyt_best.loc[i]["rank"])
    nyt_best.loc[i]["rank_last_week"] = flatten_json(nyt_best.loc[i]["rank_last_week"])
    nyt_best.loc[i]["title"] = flatten_json(nyt_best.loc[i]["title"]) 
    nyt_best.loc[i]["weeks_on_list"] = flatten_json(nyt_best.loc[i]["weeks_on_list"])     
    
nyt_best.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,5b4aa4ead3089013507db18b,2008-05-24,2008-06-08,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27,Bantam,ODD HOURS,1,0,1
1,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,25.99,"Little, Brown",THE HOST,2,1,3
2,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,24.95,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2
3,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,22.95,Putnam,THE FRONT,4,0,1
4,5b4aa4ead3089013507db18f,2008-05-24,2008-06-08,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1


In [17]:
#following is mine (CJS)
#do some cleaning and integrity checking
#to deal with typos and variant spellings

#functions for cleaning string-valued columns in dataframes
#for situations where there is a small set of valid strings
# e.g. business names, countries...
#and the strings have to belong to that set
#but there could be typos / alternate spellings
#approach:
# 1. get_matches(...) -> find set of very similar strings using Levenshtein distance
# 2. print and inspect list of matches
# 3. replace_matches(..., checked_matches) -> replace the minority value of each match with majority value

def get_matches(df, colname, levenshtein_cutoff):
    matches = []
    unique_vals = df[colname].unique()
    for i, u in enumerate(unique_vals):
        for j, v in enumerate(unique_vals):
            if(i <= j): continue
            if(lev.distance(u.lower(), v.lower())/max(len(u), len(v)) < levenshtein_cutoff):
                matches.append((u, v))
    return matches

def replace_matches(df, colname, matches):
    replacements = []
    temp_list = list(zip(*matches))
    all_match_vals = temp_list[0] + temp_list[1]
    unique_vals = np.unique(all_match_vals)
    counts = np.array([np.sum(df[colname]==v) for v in unique_vals])
    order = np.argsort(counts)[::-1]
    for ind in order:
        u = unique_vals[ind]
        to_remove = []
        for m in matches:
            if(u in m):
                other = [v for v in m if v!=u][0]
                replacements.append((other, u)) #replace <other> with u
                to_remove.append(m)
        #remove at end to avoid errors due to index shift
        for t in to_remove:
            matches.remove(t)
        #actually perform replacements here
        for r in replacements:
            df.loc[df[colname]==r[0], colname] = r[1]


In [6]:
#try to find author names that are very similar - likely typos or variant spellings
# try levenshtein distance of 0.3, turns out this is too permissive
m = get_matches(nyt_best, "author", 0.3)
for t in m: print(t)


('Dean Koontz', 'Dean R Koontz')
('Christopher Moore', 'Christopher Reich')
('Clive Cussler and Jack Du Brul', 'Clive Cussler with Jack Du Brul')
('Carol Higgins Clark', 'Mary Higgins Clark')
('Elizabeth Berg', 'Elizabeth George')
('Elizabeth Berg', 'Elizabeth Peters')
('WEB Griffin and William E Butterworth IV', 'W E B Griffin and William E Butterworth IV')
('Anne Rice', 'Luanne Rice')
('J R Ward', 'JR Ward')
('Dale Brown', 'Dan Brown')
('Stephen White', 'Stephen Hunter')
('Steve Martin', 'Steve Martini')
('JA Jance', 'J A Jance')
('Christopher Bohjalian', 'Chris Bohjalian')
('Robert Harris', 'Robert Crais')
('James Patterson and Mark Sullivan', 'James Patterson and Howard Roughan')
('James Patterson and Mark Sullivan', 'James Patterson and Marshall Karp')
('James Patterson and David Ellis', 'James Patterson and Mark Sullivan')
('Rita Mae Brown and Sneaky Pie Brown', 'Rita Mae Brown & Sneaky Pie Brown')
('Carlos Ruiz Zafon', 'Carlos Ruiz Zafón')
('James Wesley Rawles', 'James Wesley, 

In [7]:
#smaller distance -> fewer false positives
m = get_matches(nyt_best, "author", 0.15)
for t in m: print(t)

('Clive Cussler and Jack Du Brul', 'Clive Cussler with Jack Du Brul')
('WEB Griffin and William E Butterworth IV', 'W E B Griffin and William E Butterworth IV')
('J R Ward', 'JR Ward')
('Steve Martin', 'Steve Martini')
('JA Jance', 'J A Jance')
('Rita Mae Brown and Sneaky Pie Brown', 'Rita Mae Brown & Sneaky Pie Brown')
('Carlos Ruiz Zafon', 'Carlos Ruiz Zafón')
('James Wesley Rawles', 'James Wesley, Rawles')
('John le Carre', 'John le Carré')
('Jo Nesbø', 'Jo Nesbo')
('C  J Box', 'C J Box')
('Catherine Coulter and J T Ellison', 'Catherine Coulter and JT Ellison')
('Jim Harrison', 'Kim Harrison')
('Annie Proulx', 'E Annie Proulx')
('WEB Griffin and William E Butterworth', 'W E B Griffin and William E Butterworth IV')
('WEB Griffin and William E Butterworth', 'WEB Griffin and William E Butterworth IV')
('JD Robb', 'J D Robb')
('CJ Box', 'C J Box')
('BA Paris', 'B A Paris')


In [8]:
#author name matches seem all OK except for 2
m.remove(('Jim Harrison', 'Kim Harrison'))
m.remove(('Steve Martin', 'Steve Martini'))

In [9]:
#now replace the matches
replace_matches(nyt_best, 'author', m)

In [12]:
#check that this worked:
m = get_matches(nyt_best, "author", 0.15)
for t in m: print(t)

('Steve Martin', 'Steve Martini')
('Jim Harrison', 'Kim Harrison')


In [11]:
#these are precisely the two similar names we did not want to replace
#all the other cases are gone
#so everything worked

In [13]:
m = get_matches(nyt_best, "publisher", 0.3)
for t in m: print(t)

('St. Martin’s', "St. Martin's")
('Viking', 'Viking,')
('Double\xadday', 'Doubleday')
('Del Rey/ Ballantine', 'Del Rey/Ballantine')
('Ballantine,', 'Ballantine')
('Atria,', 'Atria')
('allantine', 'Ballantine')
('allantine', 'Ballantine,')
('Eos/William Morrow', 'William Morrow')
("St. Martin''s", "St. Martin's")
("St. Martin''s", 'St. Martin’s')
('Morrow/HarperCollins', 'Eos/HarperCollins')
('Harper/HarperCollins', 'Morrow/HarperCollins')
('Tor/ Tom Doherty', 'Tor/Tom Doherty')
('Del Rey/Ballantine/Lucas', 'Del Rey/Ballantine')
('Del Rey/Ballantine/Lucas', 'Del Rey/ Ballantine')
('Tor/TomDoherty', 'Tor/Tom Doherty')
('Tor/TomDoherty', 'Tor/ Tom Doherty')
('Nan. A Talese/Doubleday', 'Nan A. Talese/Doubleday')
('William Morrow/HarperCollins', 'Morrow/HarperCollins')
('St.Martin’s', "St. Martin's")
('St.Martin’s', 'St. Martin’s')
('St.Martin’s', "St. Martin''s")
('Little Brown', 'Little, Brown')
('Harper Voyager/HarperCollins', 'Harper/HarperCollins')
("Thomas Dunne/St. Martin's", 'Thomas

In [22]:
m = get_matches(nyt_best, "publisher", 0.25)
for t in m: print(t)

('St. Martin’s', "St. Martin's")
('Viking', 'Viking,')
('Double\xadday', 'Doubleday')
('Del Rey/ Ballantine', 'Del Rey/Ballantine')
('Ballantine,', 'Ballantine')
('Atria,', 'Atria')
('allantine', 'Ballantine')
('allantine', 'Ballantine,')
('ACE', 'Ace')
('Eos/William Morrow', 'William Morrow')
("St. Martin''s", "St. Martin's")
("St. Martin''s", 'St. Martin’s')
('Tor/ Tom Doherty', 'Tor/Tom Doherty')
('Tor/TomDoherty', 'Tor/Tom Doherty')
('Tor/TomDoherty', 'Tor/ Tom Doherty')
('Nan. A Talese/Doubleday', 'Nan A. Talese/Doubleday')
('St.Martin’s', "St. Martin's")
('St.Martin’s', 'St. Martin’s')
('St.Martin’s', "St. Martin''s")
('Little Brown', 'Little, Brown')
('Daw', 'DAW')
("Thomas Dunne/St. Martin's", 'Thomas Dunne/St. Martin’s')
('Del Rey/Ballantine/LucasBooks', 'Del Rey/Ballantine/Lucas')
('St. Martin‘s', "St. Martin's")
('St. Martin‘s', 'St. Martin’s')
('St. Martin‘s', "St. Martin''s")
('St. Martin‘s', 'St.Martin’s')
('Marian Wood/Putnam', 'Marion Wood/Putnam')
('Ecco/HarperCollins'

In [23]:
m.remove(('Forge/Tom Doherty', 'Tor/Tom Doherty'))
m.remove(('Forge/Tom Doherty', 'Tor/ Tom Doherty'))
m.remove(('Forge/Tom Doherty', 'Tor/TomDoherty'))
m.remove(('HarperCollins', 'Eos/HarperCollins'))
m.remove(('Ecco/HarperCollins', 'Eos/HarperCollins'))
m.remove(('Ecco/HarperCollins Publishers', 'HarperCollins Publishers'))
m.remove(('Morrow/HarperCollins Publishers', 'HarperCollins Publishers'))
m.remove(('Morrow/HarperCollins Publishers', 'Ecco/HarperCollins Publishers'))

In [24]:
for t in m: print(t)

('St. Martin’s', "St. Martin's")
('Viking', 'Viking,')
('Double\xadday', 'Doubleday')
('Del Rey/ Ballantine', 'Del Rey/Ballantine')
('Ballantine,', 'Ballantine')
('Atria,', 'Atria')
('allantine', 'Ballantine')
('allantine', 'Ballantine,')
('ACE', 'Ace')
('Eos/William Morrow', 'William Morrow')
("St. Martin''s", "St. Martin's")
("St. Martin''s", 'St. Martin’s')
('Tor/ Tom Doherty', 'Tor/Tom Doherty')
('Tor/TomDoherty', 'Tor/Tom Doherty')
('Tor/TomDoherty', 'Tor/ Tom Doherty')
('Nan. A Talese/Doubleday', 'Nan A. Talese/Doubleday')
('St.Martin’s', "St. Martin's")
('St.Martin’s', 'St. Martin’s')
('St.Martin’s', "St. Martin''s")
('Little Brown', 'Little, Brown')
('Daw', 'DAW')
("Thomas Dunne/St. Martin's", 'Thomas Dunne/St. Martin’s')
('Del Rey/Ballantine/LucasBooks', 'Del Rey/Ballantine/Lucas')
('St. Martin‘s', "St. Martin's")
('St. Martin‘s', 'St. Martin’s')
('St. Martin‘s', "St. Martin''s")
('St. Martin‘s', 'St.Martin’s')
('Marian Wood/Putnam', 'Marion Wood/Putnam')
('Little, Brown & Com

In [25]:
replace_matches(nyt_best, 'publisher', m)

In [26]:
m = get_matches(nyt_best, "title", 0.15)
for t in m: print(t)

('CROSSFIRE', 'CROSS FIRE')
('THE SINS OF THE MOTHER', 'THE SINS OF THE FATHER')
('THE FALL OF THE GOVERNOR, PART 2', 'THE FALL OF THE GOVERNOR, PART 1')
('THE BOOK OF LIFE', 'THE BOOK OF LIES')
('NYPD RED 3', 'NYPD RED 2')
('THE LAKE HOUSE', 'THE GATE HOUSE')
('NYPD RED 4', 'NYPD RED 2')
('NYPD RED 4', 'NYPD RED 3')
("THE SORCERER'S DAUGHTER", "THE MURDERER'S DAUGHTER")
('DAMAGED', 'DAMAGE')
('THE BLACK BOOK', 'THE BLACK BOX')
('THE FIX', 'THE NIX')
('THE STORE', 'THE STORM')
('THE BURNING GIRL', 'THE BURNING WIRE')
('THE FORGOTTEN ROAD', 'THE FORGOTTEN ROOM')


In [32]:
#all of these matches look like false positives, check author names:
for t in m:
    print(t[0])
    print("author:")
    print(nyt_best.loc[nyt_best['title']==t[0],"author"].values[0])
    print(t[1])
    print("author:")
    print(nyt_best.loc[nyt_best['title']==t[1],"author"].values[0])
    print("")

CROSSFIRE
author:
Dick Francis and Felix Francis
CROSS FIRE
author:
James Patterson

THE SINS OF THE MOTHER
author:
Danielle Steel
THE SINS OF THE FATHER
author:
Jeffrey Archer

THE FALL OF THE GOVERNOR, PART 2
author:
Robert Kirkman and Jay Bonansinga
THE FALL OF THE GOVERNOR, PART 1
author:
Robert Kirkman and Jay Bonansinga

THE BOOK OF LIFE
author:
Deborah Harkness
THE BOOK OF LIES
author:
Brad Meltzer

NYPD RED 3
author:
James Patterson and Marshall Karp
NYPD RED 2
author:
James Patterson and Marshall Karp

THE LAKE HOUSE
author:
Kate Morton
THE GATE HOUSE
author:
Nelson DeMille

NYPD RED 4
author:
James Patterson and Marshall Karp
NYPD RED 2
author:
James Patterson and Marshall Karp

NYPD RED 4
author:
James Patterson and Marshall Karp
NYPD RED 3
author:
James Patterson and Marshall Karp

THE SORCERER'S DAUGHTER
author:
Terry Brooks
THE MURDERER'S DAUGHTER
author:
Jonathan Kellerman

DAMAGED
author:
Lisa Scottoline
DAMAGE
author:
John Lescroart

THE BLACK BOOK
author:
James Patter

In [ ]:
#don't change any titles

In [34]:
#we want to find all unique books and assign each an ID
# bookID = <lc author>_<lc title> 

nyt_best['l_author'] = get_lc_values(nyt_best['author'])
nyt_best['l_title'] = get_lc_values(nyt_best['title'])

nyt_best['book_id'] = nyt_best['l_author']+ '_' + nyt_best['l_title']

In [37]:
m = get_matches(nyt_best, "book_id", 0.1)
for t in m: print(t)

('jamespattersonandmichaelledwidge_gone', 'jamespattersonandmichaelledwidge_zoo')
('jamespattersonandmarksullivan_privatela', 'jamespattersonandmarksullivan_privategames')
('robertkirkmanandjaybonansinga_thefallofthegovernorpart2', 'robertkirkmanandjaybonansinga_thefallofthegovernorpart1')
('jamespattersonandmarshallkarp_nypdred2', 'jamespattersonandmarshallkarp_nypdred')
('jamespattersonandmarshallkarp_nypdred3', 'jamespattersonandmarshallkarp_nypdred')
('jamespattersonandmarshallkarp_nypdred3', 'jamespattersonandmarshallkarp_nypdred2')
('jamespattersonandmarshallkarp_nypdred4', 'jamespattersonandmarshallkarp_nypdred')
('jamespattersonandmarshallkarp_nypdred4', 'jamespattersonandmarshallkarp_nypdred2')
('jamespattersonandmarshallkarp_nypdred4', 'jamespattersonandmarshallkarp_nypdred3')
('jamespattersonandmarksullivan_privateparis', 'jamespattersonandmarksullivan_privategames')
('jamespattersonandmarksullivan_privateparis', 'jamespattersonandmarksullivan_privateberlin')
('jamespatterso

In [ ]:
#book IDs seem to be OK

In [38]:
nyt_best.to_csv("nyt2_clean.csv", index=False)

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
nyt_best = pd.read_csv("nyt2_clean.csv")

In [6]:
def get_majority_string(ls):
    u = np.unique(ls)
    if(len(u)==1): return u[0]
    else: 
        counts = np.array([np.sum(np.array([s==v for s in ls])) for v in u])
        ind = np.argmax(counts)
        if(len(ind.shape)==0): return u[ind] #if ind is a scalar
        else: return u[ind[0]]

In [7]:
g = nyt_best.groupby('book_id')

summary = []

for book_id, group in g:
    entry = {}
    entry['book_id'] = book_id
    entry['title'] = group['title'].values[0]
    entry['author'] = group['author'].values[0]
    prices = np.array([float(x) for x in group['price'].values])
    ranks  = np.array([float(x) for x in group['rank'].values])
    entry['price'] = np.mean(prices[prices > 0])#can have 0 prices for unreleased books
    entry['description'] = group['description'].values[0]
    entry['publisher'] = get_majority_string(group['publisher'].values)
    entry['max_rank'] = np.max(ranks)
    entry['avg_rank'] = np.mean(ranks)
    entry['min_rank'] = np.min(ranks)
    entry['weeks_on_list'] = group.shape[0]
    dates = [datetime.strptime(d,"%Y-%m-%d") for d in group['bestsellers_date'].values]
    start_date = dates[0]
    for d in dates:
        if(d < start_date):
            start_date = d
    
    entry['start_year'] = start_date.year
    entry['start_month'] = start_date.month
    entry['start_day'] = start_date.day
    summary.append(entry)
    #completely ignore rank_last_week, because redundant and often contained spurious 0 entries
summary = pd.DataFrame(summary)

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
summary.head(10)

,book_id,title,author,price,description,publisher,max_rank,avg_rank,min_rank,weeks_on_list,start_year,start_month,start_day
0,aaronallston_backlash,BACKLASH,Aaron Allston,27.00,n the second novel in the “Star Wars: Fate of ...,Del Rey/Ballantine/Lucas,19.0,13.000000,4.0,4,2010,3,14
1,aaronallston_conviction,CONVICTION,Aaron Allston,27.00,The Jedi Order has taken control of the Galact...,Del Rey/Ballantine/Lucas,11.0,7.000000,3.0,2,2011,5,28
2,aaronallston_outcast,OUTCAST,Aaron Allston,27.00,The start of a new series about the extended S...,Del Rey/Ballantine,15.0,9.666667,3.0,3,2009,3,28
3,aaronallston_starwarsxwingmercykill,STAR WARS: X-WING: MERCY KILL,Aaron Allston,NaN,NaN,LucasBooks,18.0,18.000000,18.0,1,2012,8,11
4,abrahamverghese_cuttingforstone,CUTTING FOR STONE,Abraham Verghese,NaN,"Twin brothers, conjoined and then separated, g...",Knopf,17.0,17.000000,17.0,1,2009,2,21
5,aceatkins_robertbparkerscheapshot,ROBERT B. PARKER'S CHEAP SHOT,Ace Atkins,NaN,Spenser searches for the kidnapped son of a pl...,Putnam,18.0,15.500000,13.0,2,2014,5,10
6,aceatkins_robertbparkerskickback,ROBERT B. PARKER'S KICKBACK,Ace Atkins,NaN,Spenser defends a high school student wrongly ...,Putnam,19.0,14.500000,10.0,2,2015,5,23
7,aceatkins_robertbparkerslittlewhitelies,ROBERT B. PARKER'S LITTLE WHITE LIES,Ace Atkins,NaN,Spenser is on the trail of a con man who has d...,Putnam,11.0,11.000000,11.0,1,2017,5,6
8,aceatkins_robertbparkerslullaby,ROBERT B. PARKER’S LULLABY,Ace Atkins,26.95,Spenser helps a girl investigate her mother’s ...,Putnam,18.0,13.800000,6.0,5,2012,5,5
9,aceatkins_robertbparkersslowburn,ROBERT B. PARKER'S SLOW BURN,Ace Atkins,NaN,"Spenser is back, leaving a trail of antagonism...",Putnam,17.0,17.000000,17.0,1,2016,5,7


In [34]:
summary.to_csv("nyt_best_summary.csv", index=False)

In [143]:
_=[print(p) for p in nyt_best.loc[nyt_best['author']=="John Lescroart","description"].values]

The San Francisco defense attorney Dismas Hardy must protect the secrets of his client.
The San Francisco defense attorney Dismas Hardy must protect the secrets of his client.
A young San Francisco private investigator discovers some unpleasant facts when a well-known fund-raiser is murdered.
The San Francisco detective Abe Glitsky faces a scion of wealth who’s seeking revenge against those who sent him to prison a decade ago.
The San Francisco detective Abe Glitsky faces a wealthy heir who’s seeking revenge against those who sent him to prison.
The San Francisco detective Abe Glitsky faces a wealthy heir who is seeking revenge against those who sent him to prison.
A San Francisco private investigator discovers chilling facts about his birth family.
Dismas Hardy must defend his brother-in-law, who is accused of killing his daughter’s rapist.
Dismas Hardy and Abe Glitsky hunt for clues about a woman who has gone missing.
In the rush to find a teenager's killer, a possibly innocent middl

In [18]:
def has_keyword(df, colname, keywords):
    return [ isinstance(x, str) and any([w in x.lower() for w in keywords]) for x in df[colname].values] 

In [19]:
murder_words = [\
                "murder",\
                "murders",\
                "murdered",\
                "murderer",\
                "murderous"\
                ]

In [20]:
crime_words = [\
                       "danger", "dangerous",\
                       "chilling", "haunting",\
                       "murder", "murders", "murdered", "murderer", "murderous",\
                       "kill", "killer", "killing", "killings", "killed",\
                       "missing", "disappears", "disappeared", "disappearance", "disappearances",\
                       "vanish", "vanished", "vanishes",\
                       "violent", "violence",\
                       "crime", "crimes", "criminal",\
                       "revenge", "vengeance",\
                       "body", "bodies", "corpse", "dead", "death", "deaths"]

In [21]:
mystery_words = [\
                "mystery", "mysterious", "mysteriously", "secret", "hidden", "clue", "trail",\
                "discover", "discovers", "discovery", "discovered",\
                "uncover", "uncovers", "uncovered",
                "investigate", "investigates", "investigated", "investigator", "investigators", "investigation",\
                "detective", "sleuth"]

In [22]:
legal_words = [\
              "trial", "innocent", "convicted", "judge", "jury", "juror", "jurors"]

In [23]:
relationship_words = [\
                 "love", "loves", "lover", "lovers", "beloved"\
                 "sweetheart", "romance", "passion",\
                 "affair", "elope", "elopes", "eloped",\
                 "relationship", "relationships",\
                 "marry", "marriage", "marries",
                 "divorce", "divorces", "divorced",\
                 "fiance", "engaged", "engagement"
                ]

In [24]:
starwars_words = ['jedi', 'sith', 'star wars', 'galactic empire', 'galactic alliance', 'skywalker', 'rebel alliance' ]

In [33]:
summary["keyword_crime"] = has_keyword(summary, 'description', crime_words)
summary["keyword_murder"] = has_keyword(summary, 'description', murder_words)
summary["keyword_mystery"] = has_keyword(summary, 'description', mystery_words)
summary["keyword_relationship"] = has_keyword(summary, 'description', relationship_words)
summary["keyword_newyork"] = has_keyword(summary, 'description', ['new york'])
summary["keyword_starwars"] = has_keyword(summary, 'description', starwars_words)

In [39]:
#some final cleaning based on inspection in Tableau
summary.loc[summary['publisher']=="Atlantic Monthly Press", "publisher"] = "Atlantic Monthly"
summary.loc[summary['publisher']=="Delacorte Press", "publisher"] = "Delacorte"
summary.loc[summary['publisher']=="Doubleday Publishing", "publisher"] = "Doubleday"
summary.loc[summary['publisher']=="Ecco/HarperCollins Publishers", "publisher"] = "Ecco/HarperCollins"
summary.loc[summary['publisher']=="Flatiron Books", "publisher"] = "Flatiron"
summary.loc[summary['publisher']=="Gallery Books", "publisher"] = "Gallery"
summary.loc[summary['publisher']=="Pocket Books", "publisher"] = "Pocket"
summary.loc[summary['publisher']=="St. Martin's Press", "publisher"] = "St. Martin's"


In [40]:
summary.to_csv("nyt_best_summary.csv", index=False)

In [42]:
summary_sorted = summary.sort_values(by="weeks_on_list", axis=0, ascending=False)

In [44]:
summary_sorted.head(20)

,book_id,title,author,price,description,publisher,max_rank,avg_rank,min_rank,weeks_on_list,start_year,start_month,start_day,keyword_crime,keyword_murder,keyword_mystery,keyword_relationship,keyword_newyork,keyword_starwars
148,anthonydoerr_allthelightwecannotsee,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,NaN,The lives of a blind French girl and a gadget-...,Scribner,20.0,7.283688,1.0,141,2014,5,10,False,False,False,False,False,False
1401,kathrynstockett_thehelp,THE HELP,Kathryn Stockett,24.95,A young white woman and two black maids collab...,Amy Einhorn/Putnam,19.0,7.122807,1.0,114,2009,3,14,False,False,False,False,False,False
756,georgerrmartin_adancewithdragons,A DANCE WITH DRAGONS,George R R Martin,35.00,"After a colossal battle, the Seven Kingdoms fa...",Bantam,20.0,11.981132,1.0,106,2011,7,16,False,False,False,False,False,False
762,gillianflynn_gonegirl,GONE GIRL,Gillian Flynn,25.00,A woman disappears on the day of her fifth ann...,Crown,20.0,6.712644,1.0,87,2012,6,9,True,False,False,False,False,False
1455,kristinhannah_thenightingale,THE NIGHTINGALE,Kristin Hannah,NaN,Two sisters in World War II France: one strugg...,St. Martin's,20.0,9.724138,3.0,87,2015,2,7,False,False,False,False,False,False
2111,stieglarsson_thegirlwhokickedthehornetsnest,THE GIRL WHO KICKED THE HORNET’S NEST,Stieg Larsson,27.95,The third volume of a trilogy about a Swedish ...,Knopf,17.0,6.225000,1.0,80,2010,5,30,False,False,False,False,False,False
1813,paulahawkins_thegirlonthetrain,THE GIRL ON THE TRAIN,Paula Hawkins,NaN,A psychological thriller set in London is full...,Riverhead,19.0,4.587500,1.0,80,2015,1,17,False,False,False,False,False,False
614,donnatartt_thegoldfinch,THE GOLDFINCH,Donna Tartt,NaN,A painting smuggled out of the Metropolitan M...,"Little, Brown",20.0,7.514706,1.0,68,2013,10,26,False,False,False,False,False,False
2074,stepheniemeyer_thehost,THE HOST,Stephenie Meyer,25.99,Aliens have taken control of the minds and bod...,"Little, Brown",19.0,8.102941,2.0,68,2008,5,24,True,False,False,False,False,False
79,amortowles_agentlemaninmoscow,A GENTLEMAN IN MOSCOW,Amor Towles,NaN,A Russian count undergoes 30 years of house ar...,Viking,19.0,12.000000,6.0,63,2016,9,10,False,False,False,False,False,False


In [25]:
np.mean(has_keyword(summary, 'description', murder_words))

0.08891752577319588

In [26]:
np.mean(has_keyword(summary, 'description', starwars_words))

0.01245704467353952

In [27]:
np.mean(has_keyword(summary, 'description', crime_words))

0.2847938144329897

In [28]:
np.mean(has_keyword(summary, 'description', mystery_words))

0.24527491408934707

In [29]:
np.mean(np.logical_and(has_keyword(summary, 'description', crime_words), has_keyword(summary, 'description', mystery_words)))

0.12800687285223367

In [30]:
np.mean(has_keyword(summary, 'description', legal_words))

0.011597938144329897

In [31]:
np.mean(has_keyword(summary, 'description', relationship_words))

0.07689003436426117

In [32]:
np.mean(has_keyword(summary, 'description', ["new york"]))

0.033934707903780066

In [100]:
fantasy_words = [\
                "magic", "curse", "sorcery"\
                "witch", "witches",\
                "wizard", "wizards",\
                "sorcerer", "sorcerers",\
                "spell", "spells",\
                "dragon", "dragons",\
              ]

In [73]:
keyword_the = [any([m in x.lower() for m in ['the']]) for x in summary['description'].values] 

In [101]:
np.mean(has_keyword(summary, 'description', fantasy_words))

0.013745704467353952

In [102]:
np.mean(has_keyword(summary, 'description', ["mysterious"]))

0.01804123711340206

In [75]:
keyword_the

[True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 Tru

In [ ]:
ls = [(1,2), (2, 'a')]
x = ls[1]
ls.remove(x)
ls

In [ ]:
#check for IDs that are similar
#because these may be spurious duplications from the same book
#that arise due to typo
possible_matches = []
book_ID_groups = raw_books.groupby('book_ID')
for i, (ID1, fields1) in enumerate(book_ID_groups):
    for j, (ID2, fields2) in enumerate(book_ID_groups):
        if(j <= i): continue
        if(lev.distance(ID1, ID2)/max(len(ID1), len(ID2)) < 0.1):
            print("Possible match:")
            print(ID1)
            print(ID2)
            possible_matches.append((ID1, ID2))
    print("Finished checking "+ID1)
print("Finished checking all IDs")

In [ ]:
#can hand-curate possible matches here...

for (ID1, ID2) in possible_matches:
    (n1, n2) = (np.sum(raw_books['book_ID']==ID) for ID in (ID1, ID2))
    if(n1 > n2): raw_books.loc[raw_books['book_ID']==ID2, 'book_ID'] = ID1
           else: raw_books.loc[raw_books['book_ID']==ID1, 'book_ID'] = ID2

In [ ]:
#now we have hopefully gathered the correct set of entries under the correct book_ID


In [ ]:
np.array([3,2]).shape

In [ ]:
#do integrity checking on publishers
unique_publishers = raw_books['publisher'].unique()
possible_matches = []
for i, p1 in enumerate(unique_publishers):
    for j, p2 in enumerate(unique_publishers):
        if(i <= j): continue
        if(lev.distance(p1, p2)/max(len(p1), len(p2)) < 0.15):
            print("Possible match:")
            print(p1)
            print(p2)
            possible_matches.append((p1, p2))

In [ ]:
#replace with majority
#go through publishers in order of size (number of records)
#and take all matches
replacements = []
publisher_counts = np.array([np.sum(raw_books["publisher"]==p) for p in unique_publishers])
order = np.argsort(publisher_counts)
for ind in order:
    p1 = unique_publishers[ind]
    to_remove = []
    for t in possible_matches:
        if(p1 in t):
            other = [u for u in t if u!=p1]
            replacements.append((other, p1))
            to_remove.append((other, p1))
    for t in to_remove:
        possible_matches.remove(t)
#actually perform replacements here
for t in replacements:
    raw_books.loc[raw_books['publisher']==t[0], 'publisher'] = t[1]
    

In [ ]:
get_majority_string(['stuff', 'x', 'stuff', 'y', 'x', 'x'])

In [ ]:
print('Total number of titles: %s' % (len(raw_books['title'].unique())))
print('Total number of clean titles: %s' % (len(raw_books['l_title'].unique())))

In [ ]:
raw_books.head()

In [ ]:
for gi in g: print(gi)

In [ ]:
print(raw_books['title'].unique())
print(raw_books['l_title'].unique())

In [ ]:
#find just one clean lower title in data that has two different real titles

spec_title = raw_books.groupby(['l_title', 'title'])

doubleTitle = ''
for name, group in spec_title:
    if doubleTitle == name[0]:
        print('lower title: %s' % (name[0]))
    doubleTitle = name[0]
    


In [ ]:
#find real titles based on single lower case title
filter_titles = raw_books[(raw_books['l_title'] == 'crossfire')].title.unique()
for t in filter_titles:
    print(raw_books[(raw_books['title'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

In [ ]:
#so we can conclude that title is clean, there is one case where lc stripped version is the same

In [ ]:
print('Total number of authors: %s' % (len(raw_books['author'].unique())))
print('Total number of clean authors: %s' % (len(raw_books['l_author'].unique())))

In [ ]:
spec_author = raw_books.groupby(['l_author', 'author'])

double_author = ''
for name, group in spec_author:
    if double_author == name[0]:
        print('lower author: %s' % (name[0]))
    double_author = name[0]

In [ ]:
#find real author name based on single lower case author name
filter_auth_col = raw_books[(raw_books['l_author'] == 'baparis')].author.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['author'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

In [ ]:

print('Total number of publisher names: %s' % (len(raw_books['publisher'].unique())))
print('Total number of clean publisher names: %s' % (len(raw_books['l_publisher'].unique())))

In [ ]:
spec_publisher = raw_books.groupby(['l_publisher', 'publisher'])

double_publisher = ''
for name,group in spec_publisher:
    if double_publisher == name[0]:
        print(name[0])
    double_publisher = name[0]


In [ ]:


filter_auth_col = raw_books[(raw_books['l_publisher'] == 'stmartins')].publisher.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['publisher'] == t)][['publisher']].reset_index(drop=True)[:1])

